# 🌟 **NER (Named Entity Recognition)**

NER is a task in NLP where the model **finds important real-world entities** in text and assigns labels like:
PERSON, ORG (organization), GPE (countries, cities, states), DATE, MONEY, PRODUCT, etc.

Example:
**“Apple released the iPhone in California.”**

| Word       | Entity  |
| ---------- | ------- |
| Apple      | ORG     |
| iPhone     | PRODUCT |
| California | GPE     |

---

### ⭐ Why NER is used

* Resume parsing
* Chatbots
* Search engines
* Medical reports
* Invoice extraction
* News understanding
* Legal document automation

---

### ⭐How NER Works

NER uses:

✔ **Rule-based NER**

Uses patterns (regex, dictionaries).
Simple but weak.

✔ **Statistical / ML-based NER**

SVM, CRF (old but used in classic NLP).

✔ **Neural NER (current trend)**

spaCy, BERT, RoBERTa, transformers → highest accuracy.

---

# ⭐ Basic NER using spaCy (Built-in Model)

In [ ]:
pip install spacy

In [ ]:
python -m spacy download en_core_web_sm

In [7]:
import spacy

nlp = spacy.load("en_core_web_sm")

text = "Google hired John in London for 5 million dollars."

doc = nlp(text)

for ent in doc.ents:
    print(ent.text,"-", ent.label_)

Google - ORG
John - PERSON
London - GPE
5 million dollars - MONEY


In [4]:
# spaCy NER Visualization (Optional)

from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

### ⭐ Understanding Common NER Labels

| Label   | Meaning              |
| ------- | -------------------- |
| PERSON  | Human names          |
| ORG     | Company, institution |
| GPE     | Countries, cities    |
| LOC     | Locations            |
| DATE    | Dates                |
| TIME    | Times                |
| MONEY   | Monetary values      |
| PRODUCT | Device, product      |
| EVENT   | Festive/event names  |

---

#### spaCy NER may be inaccurate

Because:

* Small training data
* “sm” model is small
* Domain-mismatch
* New/unseen entities
* Not trained on Indian names/places well

**Fix:**
Use `en_core_web_trf` (transformer).

```bash
python -m spacy download en_core_web_trf
```

## When to use Transformers (BERT NER)

If your requirements are:

* High accuracy
* Domain-specific text
* Uncommon entities
* Large labels

Use HuggingFace:

In [ ]:
pip install transformers

### Simple NER using BERT model

In [8]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)

text = "Elon Musk founded SpaceX in California."

result = ner(text)

c:\Users\Admin\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
c:\Users\Admin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:  79%|#######9  | 1.06G/1.33G [00:00<?, ?B/s]

c:\Users\Admin\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--dbmdz--bert-large-cased-finetuned-conll03-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were n

In [7]:
print(result)

[{'entity_group': 'PER', 'score': 0.99840075, 'word': 'Elon Musk', 'start': 0, 'end': 9}, {'entity_group': 'ORG', 'score': 0.9987556, 'word': 'SpaceX', 'start': 18, 'end': 24}, {'entity_group': 'LOC', 'score': 0.99960726, 'word': 'California', 'start': 28, 'end': 38}]


# ⭐ Training Custom NER (Simple Version)

When default NER doesn’t detect what you want, you train a custom model.


### Example: Label **“DEVICE”** for text like “Redmi Note 12”.

## Step 1: Prepare Training Data

In [12]:
TRAIN_DATA = [
    ("I bought a Redmi Note 12 yesterday.", 
     {"entities": [(11, 25, "DEVICE")]}),

    ("The Samsung S22 Ultra is expensive.", 
     {"entities": [(4, 20, "DEVICE")]}),

    ("Xiaomi 14 Pro is a flagship phone.", 
     {"entities": [(0, 13, "DEVICE")]}),

    ("I am planning to buy the Xiaomi 14 Pro.", 
     {"entities": [(27, 40, "DEVICE")]}),

    ("OnePlus 12 is launching next month.", 
     {"entities": [(0, 10, "DEVICE")]}),
]


## Step 2: Train Model

In [13]:
import spacy
from spacy.training.example import Example

# ---------------------------------------------------------
# 1. Create a blank English NLP pipeline
# ---------------------------------------------------------
# Using blank("en") because we want to train NER from scratch,
# not use the default pretrained model.
nlp = spacy.blank("en")

# ---------------------------------------------------------
# 2. Add NER component to the pipeline
# ---------------------------------------------------------
# "ner" = Named Entity Recognizer
ner = nlp.add_pipe("ner")

# ---------------------------------------------------------
# 3. Add the new custom label we want the model to detect
# ---------------------------------------------------------
ner.add_label("DEVICE")  # Example: Redmi Note 12, Samsung S22

# ---------------------------------------------------------
# 4. Initialize the model
# ---------------------------------------------------------
# This sets up the model weights based on the labels we added.
optimizer = nlp.initialize()

# ---------------------------------------------------------
# 5. Start training loop
# ---------------------------------------------------------
# We train for several epochs (iterations).
# More epochs = better learning, but too many = overfitting.
for epoch in range(20):
    print(f"Epoch {epoch+1} started...")
    
    # Loop through each training example
    for text, annotations in TRAIN_DATA:

        # Step 1: Convert text into a spaCy doc object
        doc = nlp.make_doc(text)

        # Step 2: Convert your annotation dict into a training Example object
        example = Example.from_dict(doc, annotations)

        # Step 3: Update the NER model with this example
        # sgd = stochastic gradient descent (optimizer)
        nlp.update([example], sgd=optimizer)

print("Training completed!")


Epoch 1 started...


c:\Users\Admin\anaconda3\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "I am planning to buy the Xiaomi 14 Pro." with entities "[(27, 40, 'DEVICE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Epoch 2 started...
Epoch 3 started...
Epoch 4 started...
Epoch 5 started...
Epoch 6 started...
Epoch 7 started...
Epoch 8 started...
Epoch 9 started...
Epoch 10 started...
Epoch 11 started...
Epoch 12 started...
Epoch 13 started...
Epoch 14 started...
Epoch 15 started...
Epoch 16 started...
Epoch 17 started...
Epoch 18 started...
Epoch 19 started...
Epoch 20 started...
Training completed!


## Step 3: Test Model

In [14]:
test_text = "Xiaomi 14 Pro is the latest model."
doc = nlp(test_text)

for ent in doc.ents:
    print(ent.text, ent.label_)

Xiaomi 14 Pro DEVICE


**NOTE:**  
* Minimum **200–500 labelled sentences** for reliable performance
* Ensure entity boundaries are correct
* Mix short + long sentences
* Include variations of names
* Add negative examples
* Avoid overfitting by adding diverse sentences